In [2]:
# model 1 把所有normalize換成 batch normalize, 所有nonlinear 換成leaky relu, 在fpn最前面加入norm並且在所有相加的部分前加入norm
import torch
import cv2
import os
import numpy as np
import shutil
from google.colab.patches import cv2_imshow
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import math
from PIL import Image
import torch.nn as nn
import yaml
import random
from google.colab import files
import sys
import time
from torch.utils.data import random_split
device = "cuda" if torch.cuda.is_available() else "cpu" # 檢查是否有可用的 CUDA 設備（通常是顯卡，支援 GPU 運算），如果有，就將 device 變數設置為 "cuda"，否則設置為 "cpu"。

In [3]:
# check = {"test" : 2}

# torch.save(checkpoint, 'test1.pth')
# source_path = 'test1.pth'
# destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip_shuffle'
# shutil.copy(source_path, destination_path)

In [4]:
# check2 = torch.load('/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip_shuffle/test1.pth')
# check2['test']

In [5]:
# ori_data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip_shuffle'
# ori_data_path = ori_data_path + '/' + 'dataset_1.pth'
# check = torch.load(ori_data_path)


In [6]:
# print(check['depth_name'][:20])
# print(check['depth_name'][-20:])

In [7]:
# # shuffle dataset
# data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip_shuffle/dataset_1.pth_2'

# new_file_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip_shuffle/dataset_1.pth_3'
# # os.rename(data_path, new_file_path)
# data_path = new_file_path
# os.remove(data_path)


In [ ]:
# shuffle dataset
start_time = time.time()
how_many_times = 1000
for count in range(how_many_times):
    ori_data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip_shuffle'
    ori_name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip_shuffle'

    ori_data_path_list = sorted(os.listdir(ori_data_path))
    ori_name_path_list = sorted(os.listdir(ori_name_path))
    ori_length = len(ori_data_path_list)

    # path = os.path.join(ori_data_path, ori_data_path_list[0])
    random_number1 = random.randint(0, ori_length - 1)
    random_number2 = random_number1
    while(random_number2 == random_number1):
        random_number2 = random.randint(0, ori_length - 1)

    shuffle_data_path1 = os.path.join(ori_data_path, ori_data_path_list[random_number1])
    shuffle_data_path2 = os.path.join(ori_data_path, ori_data_path_list[random_number2])
    shuffle_name_path1 = os.path.join(ori_name_path, ori_name_path_list[random_number1])
    shuffle_name_path2 = os.path.join(ori_name_path, ori_name_path_list[random_number2])

    data_check1 = torch.load(shuffle_data_path1)
    data_check2 = torch.load(shuffle_data_path2)
    name_check1 = torch.load(shuffle_name_path1)
    name_check2 = torch.load(shuffle_name_path2)


    image1 = data_check1['image_list']
    depth1 = data_check1['depth_list']
    image_name1 = name_check1['image_name']
    depth_name1  = name_check1['depth_name']
    # print(image_name1[:5])
    image2 = data_check2['image_list']
    depth2 = data_check2['depth_list']
    image_name2 = name_check2['image_name']
    depth_name2 = name_check2['depth_name']
    start = 0
    end = 4999
    num_samples = 5000
    random_list1 = random.sample(range(start, end + 1), num_samples)
    random_list2 = random.sample(range(start, end + 1), num_samples)

    image1_tmp = []
    for idx in random_list1:
        image1_tmp.append(image1[idx])
    image1 = image1_tmp
    del image1_tmp

    depth1_tmp = []
    for idx in random_list1:
        depth1_tmp.append(depth1[idx])
    depth1 = depth1_tmp
    del depth1_tmp

    image_name1_tmp = []
    for idx in random_list1:
        image_name1_tmp.append(image_name1[idx])
    image_name1 = image_name1_tmp
    del image_name1_tmp

    depth_name1_tmp = []
    for idx in random_list1:
        depth_name1_tmp.append(depth_name1[idx])
    depth_name1 = depth_name1_tmp
    del depth_name1_tmp

    image2_tmp = []
    for idx in random_list2:
        image2_tmp.append(image2[idx])
    image2 = image2_tmp
    image2_tmp = image2_tmp[:2500]

    depth2_tmp = []
    for idx in random_list2:
        depth2_tmp.append(depth2[idx])
    depth2 = depth2_tmp
    depth2_tmp = depth2_tmp[:2500]

    image_name2_tmp = []
    for idx in random_list2:
        image_name2_tmp.append(image_name2[idx])
    image_name2 = image_name2_tmp
    image_name2_tmp = image_name2_tmp[:2500]

    depth_name2_tmp = []
    for idx in random_list2:
        depth_name2_tmp.append(depth_name2[idx])
    depth_name2 = depth_name2_tmp
    depth_name2_tmp = depth_name2_tmp[:2500]

    image2[:2500] = image1[:2500]
    depth2[:2500] = depth1[:2500]
    depth_name2[:2500] = depth_name1[:2500]
    image_name2[:2500] = image_name1[:2500]

    image1[:2500] = image2_tmp
    depth1[:2500] = depth2_tmp
    image_name1[:2500] = image_name2_tmp
    depth_name1[:2500] = depth_name2_tmp

    print(len(image_name1))
    print(image_name1[-5:])


    # print(image_name1[:5])

    data_check1 = {
                'depth_list': depth1,
                'image_list' : image1,
            }
    name_check1 = {
                'depth_name' : depth_name1,
                'image_name' : image_name1
            }

    data_check2 = {
                'depth_list': depth2,
                'image_list' : image2,
            }
    name_check2 = {
                'depth_name' : depth_name2,
                'image_name' : image_name2
            }

    # os.remove(shuffle_data_path1)
    # os.remove(shuffle_data_path2)
    # os.remove(shuffle_name_path1)
    # os.remove(shuffle_name_path2)

    torch.save(data_check1, ori_data_path_list[random_number1])
    source_path = ori_data_path_list[random_number1]
    destination_path = ori_data_path
    shutil.copy(source_path, destination_path)


    torch.save(data_check2, ori_data_path_list[random_number2])
    source_path = ori_data_path_list[random_number2]
    destination_path = ori_data_path
    shutil.copy(source_path, destination_path)

    torch.save(name_check1, ori_name_path_list[random_number1])
    source_path = ori_name_path_list[random_number1]
    destination_path = ori_name_path
    shutil.copy(source_path, destination_path)

    torch.save(name_check2, ori_name_path_list[random_number2])
    source_path = ori_name_path_list[random_number2]
    destination_path = ori_name_path
    shutil.copy(source_path, destination_path)




    print(random_number1, random_number2)
end_time = time.time()
elapsed_time = end_time - start_time
print("代码段执行时间:", elapsed_time, "秒")
from google.colab import runtime
runtime.unassign()

5000
['/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_28_drive_0039_sync/image_02/data/0000000268.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_28_drive_0162_sync/image_03/data/0000000032.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_26_drive_0019_sync/image_03/data/0000000087.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_10_03_drive_0027_sync/image_03/data/0000000054.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_26_drive_0061_sync/image_03/data/0000000429.png']
8 0
5000
['/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0028_sync/image_03/data/0000001151.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_26_drive_0101_sync/image_03/data/0000000040.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0033_sync/image_03/data/0000001332.png', '/content/drive/MyDrive/Colab No

In [ ]:
'''
# shuffle dataset

image1 = data_check1['image_list']
depth1 = data_check1['depth_list']
image_name1 = name_check1['image_name']
depth_name1  = name_check1['depth_name']
# print(image_name1[:5])
image2 = data_check2['image_list']
depth2 = data_check2['depth_list']
image_name2 = name_check2['image_name']
depth_name2 = name_check2['depth_name']
start = 0
end = 4999
num_samples = 5000
random_list1 = random.sample(range(start, end + 1), num_samples)
random_list2 = random.sample(range(start, end + 1), num_samples)

image1_tmp = []
for idx in random_list1:
    image1_tmp.append(image1[idx])
image1 = image1_tmp
del image1_tmp

depth1_tmp = []
for idx in random_list1:
    depth1_tmp.append(depth1[idx])
depth1 = depth1_tmp
del depth1_tmp

image_name1_tmp = []
for idx in random_list1:
    image_name1_tmp.append(image_name1[idx])
image_name1 = image_name1_tmp
del image_name1_tmp

depth_name1_tmp = []
for idx in random_list1:
    depth_name1_tmp.append(depth_name1[idx])
depth_name1 = depth_name1_tmp
del depth_name1_tmp

image2_tmp = []
for idx in random_list2:
    image2_tmp.append(image2[idx])
image2 = image2_tmp
image2_tmp = image2_tmp[:2500]

depth2_tmp = []
for idx in random_list2:
    depth2_tmp.append(depth2[idx])
depth2 = depth2_tmp
depth2_tmp = depth2_tmp[:2500]

image_name2_tmp = []
for idx in random_list2:
    image_name2_tmp.append(image_name2[idx])
image_name2 = image_name2_tmp
image_name2_tmp = image_name2_tmp[:2500]

depth_name2_tmp = []
for idx in random_list2:
    depth_name2_tmp.append(depth_name2[idx])
depth_name2 = depth_name2_tmp
depth_name2_tmp = depth_name2_tmp[:2500]

image2[:2500] = image1[:2500]
depth2[:2500] = depth1[:2500]
depth_name2[:2500] = depth_name1[:2500]
image_name2[:2500] = image_name1[:2500]

image1[:2500] = image2_tmp
depth1[:2500] = depth2_tmp
image_name1[:2500] = image_name2_tmp
depth_name1[:2500] = depth_name2_tmp

print(len(image_name1))
print(image_name1[-5:])


# print(image_name1[:5])

data_check1 = {
              'depth_list': depth1,
              'image_list' : image1,
          }
name_check1 = {
              'depth_name' : depth_name1,
              'image_name' : image_name1
          }

data_check2 = {
              'depth_list': depth2,
              'image_list' : image2,
          }
name_check2 = {
              'depth_name' : depth_name2,
              'image_name' : image_name2
          }

os.remove(shuffle_data_path1)
os.remove(shuffle_data_path2)
os.remove(shuffle_name_path1)
os.remove(shuffle_name_path2)

torch.save(data_check1, ori_data_path_list[random_number1])
source_path = ori_data_path_list[random_number1]
destination_path = ori_data_path
shutil.copy(source_path, destination_path)


torch.save(data_check2, ori_data_path_list[random_number2])
source_path = ori_data_path_list[random_number2]
destination_path = ori_data_path
shutil.copy(source_path, destination_path)

torch.save(name_check1, ori_name_path_list[random_number1])
source_path = ori_name_path_list[random_number1]
destination_path = ori_name_path
shutil.copy(source_path, destination_path)

torch.save(name_check2, ori_name_path_list[random_number2])
source_path = ori_name_path_list[random_number2]
destination_path = ori_name_path
shutil.copy(source_path, destination_path)
'''

5000
['/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0034_sync/image_02/data/0000000394.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0033_sync/image_03/data/0000000959.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0034_sync/image_03/data/0000000030.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0034_sync/image_03/data/0000000051.png', '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image/2011_09_30_drive_0034_sync/image_03/data/0000000441.png']


'/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip_shuffle/dataset_10.pth'

In [ ]:
# new_shuffle_data_path1 = shuffle_data_path1 + '2'
# new_shuffle_data_path2 = shuffle_data_path2 + '2'
# new_shuffle_name_path1 = shuffle_name_path1 + '2'
# new_shuffle_name_path2 = shuffle_name_path2 + '2'

/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip_shuffle/dataset_13.pth2


In [ ]:
list1 = [1, 2, 3, 4]
list2 = [5, 6, 7, 8]
list1[:2] = list2[:2]
list1
list1 = [1, 2, 3, 4]
list1 = list1[:2]
list1

[1, 2]

In [ ]:
def deb(param, str):
  print(str + " = {}".format(param))

In [ ]:
def load_config(file_path):
    with open(file_path, 'r') as file:
        config = yaml.safe_load(file)
    return config

In [ ]:
# path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Model/depth_analysis.pth' # 讀取depth的統計數字
path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Model/depth_analysis.pth'

check = torch.load(path)
total_sum = check['total_sum']
DEPTH_NONZERO = check['total_nonzero']
DEPTH_MEAN = check['total_mean']
DEPTH_STD = check['total_std']
del check

In [ ]:
def count_params(model):
  sum = 0
  for param in model.parameters():
    sum = sum + param.numel()
  return sum

In [ ]:
# file_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Model/config.yml'
file_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Model/config.yml'
config = load_config(file_path)

In [ ]:
target_size = (config['data']['image_size'], config['data']['image_size'])

In [ ]:
'''
def find_next_file(first, second):
  first_list = sorted(os.listdir(first))
  second_list = sorted(os.listdir(second))
  length = len(first_list)
  while True:
    rand_int = random.randint(0, length - 1)
    target = first_list[rand_int]
    if target in second_list:
      return first + '/' + target, second + '/' + target
'''

"\ndef find_next_file(first, second):\n  first_list = sorted(os.listdir(first))\n  second_list = sorted(os.listdir(second))\n  length = len(first_list)\n  while True:\n    rand_int = random.randint(0, length - 1)\n    target = first_list[rand_int]\n    if target in second_list:\n      return first + '/' + target, second + '/' + target\n"

In [ ]:
'''
def compute_depth_mean(path):
  file_list = sorted(os.listdir(path))
  total_sum = 0

  total_nonzero = 0
  # count = 0
  for name in file_list:
    file_path = path + '/' + name
    check = torch.load(file_path)

    target = torch.tensor(check['depth_list']).to(torch.float64)
    print(target.dtype)
    total_sum += torch.sum(target)

    total_nonzero += torch.nonzero(target).size(0)
    # count += 1
    # if count == 2:
    #   break
  total_mean = total_sum / total_nonzero
  return total_sum, total_nonzero, total_mean


'''

"\ndef compute_depth_mean(path):\n  file_list = sorted(os.listdir(path))\n  total_sum = 0\n\n  total_nonzero = 0\n  # count = 0\n  for name in file_list:\n    file_path = path + '/' + name\n    check = torch.load(file_path)\n\n    target = torch.tensor(check['depth_list']).to(torch.float64)\n    print(target.dtype)\n    total_sum += torch.sum(target)\n\n    total_nonzero += torch.nonzero(target).size(0)\n    # count += 1\n    # if count == 2:\n    #   break\n  total_mean = total_sum / total_nonzero\n  return total_sum, total_nonzero, total_mean\n\n\n"

In [ ]:
'''
path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
total_sum, total_nonzero, total_mean = compute_depth_mean(path)
print(total_sum, total_nonzero, total_mean)
'''

"\npath = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'\ntotal_sum, total_nonzero, total_mean = compute_depth_mean(path)\nprint(total_sum, total_nonzero, total_mean)\n"

In [ ]:
'''
def compute_depth_std(path, total_mean, total_nonzero):
  file_list = sorted(os.listdir(path))


  total_sum = 0
  # count = 0
  for name in file_list:
    file_path = path + '/' + name
    check = torch.load(file_path)

    target = torch.tensor(check['depth_list']).to(torch.float64)
    non_zero_mask = target != 0

    target = target[non_zero_mask]
    target = target - total_mean
    target = target ** 2
    target = target / (total_nonzero - 1)
    total_sum += torch.sum(target)


    # count += 1
    # if count == 2:
    #   break

  return total_sum
'''



"\ndef compute_depth_std(path, total_mean, total_nonzero):\n  file_list = sorted(os.listdir(path))\n\n\n  total_sum = 0\n  # count = 0\n  for name in file_list:\n    file_path = path + '/' + name\n    check = torch.load(file_path)\n\n    target = torch.tensor(check['depth_list']).to(torch.float64)\n    non_zero_mask = target != 0\n\n    target = target[non_zero_mask]\n    target = target - total_mean\n    target = target ** 2\n    target = target / (total_nonzero - 1)\n    total_sum += torch.sum(target)\n\n\n    # count += 1\n    # if count == 2:\n    #   break\n\n  return total_sum\n"

In [ ]:
'''
std_square = compute_depth_std(path, total_mean, total_nonzero)
print(std_square)
'''

'\nstd_square = compute_depth_std(path, total_mean, total_nonzero)\nprint(std_square)\n'

In [ ]:
'''
checkpoint = {
          'total_sum': total_sum,
          'total_nonzero': total_nonzero, # model.state_dict()是存下param的的值和形狀
          'total_mean': total_mean, # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
          'total_std' : total_std
        }

torch.save(checkpoint, 'depth_analysis2.pth')
source_path = 'depth_analysis2.pth'
destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Model'
shutil.copy(source_path, destination_path)
'''


"\ncheckpoint = {\n          'total_sum': total_sum,\n          'total_nonzero': total_nonzero, # model.state_dict()是存下param的的值和形狀\n          'total_mean': total_mean, # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度\n          'total_std' : total_std\n        }\n\ntorch.save(checkpoint, 'depth_analysis2.pth')\nsource_path = 'depth_analysis2.pth'\ndestination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Model'\nshutil.copy(source_path, destination_path)\n"

In [ ]:
'''
def find_next_file(first, second):
  first_list = sorted(os.listdir(first))
  second_list = sorted(os.listdir(second))
  length = len(first_list)
  while True:
    rand_int = random.randint(0, length - 1)
    target = first_list[rand_int]
    if target in second_list:
      return first + '/' + target, second + '/' + target
'''

"\ndef find_next_file(first, second):\n  first_list = sorted(os.listdir(first))\n  second_list = sorted(os.listdir(second))\n  length = len(first_list)\n  while True:\n    rand_int = random.randint(0, length - 1)\n    target = first_list[rand_int]\n    if target in second_list:\n      return first + '/' + target, second + '/' + target\n"

In [ ]:
'''
def create_dataset_save(first_path, second_path, epoch_size = 5000): # first is depth, second is image
  depth_list = []
  image_list = []
  depth_name_list = []
  image_name_list = []
  count = 0
  file_count = 8  #改

  first_path_list1 = sorted(os.listdir(first_path))
  second_path_list1 = sorted(os.listdir(second_path))
  idx1_start = 109   #改

  for idx1 in range(idx1_start, len(first_path_list1)):

    first_seq1 = first_path_list1[idx1]
    # if first_seq1 == '2011_09_28_drive_0090_sync': #改
    #   continue
    first_path2 = first_path + '/' + first_seq1 + '/proj_depth/groundtruth'
    second_path2 = second_path + '/' + first_seq1
    first_path_list2 = sorted(os.listdir(first_path2))

    if idx1 == idx1_start:
      idx2_start = 1  # 改
    else:
      idx2_start = 0
    for idx2 in range(idx2_start, len(first_path_list2)): # image02, image03
      first_seq2 = first_path_list2[idx2]
      first_path3 = first_path2 + '/' + first_seq2
      second_path3 = second_path2 + '/' + first_seq2 + '/data'
      first_path_list3 = sorted(os.listdir(first_path3))
      if idx1 == idx1_start and idx2 == idx2_start:
        idx3_start = 58 #改成+1
      else:
        idx3_start = 0
      for idx3 in range(idx3_start, len(first_path_list3)):
        first_seq3 = first_path_list3[idx3]
        first_path4 = first_path3 + '/' + first_seq3
        second_path4 = second_path3 + '/' + first_seq3
        print(count)
        print(first_path4)
        print(second_path4)
        if first_path4[-5] == ')':
          continue
        depth = np.array(Image.open(first_path4), dtype=np.int16)
        depth = cv2.resize(depth, target_size, cv2.INTER_LANCZOS4)
        depth = depth.astype(np.float16) / 256.0


        image = cv2.imread(second_path4)


        image = cv2.resize(image, target_size, interpolation = cv2.INTER_LANCZOS4)
        depth_list.append(depth)
        depth_name_list.append(first_path4)
        image_list.append(image)
        image_name_list.append(second_path4)
        count = count + 1


        if count == epoch_size:
          count = 0
          checkpoint = {
              'depth_list': depth_list,
              'image_list' : image_list,
          }
          checkpoint2 = {
              'depth_name' : depth_name_list,
              'image_name' : image_name_list
          }
          name = 'dataset_{}.pth'.format(file_count)
          name2 = 'name_list_{}.pth'.format(file_count)
          file_count = file_count + 1
          torch.save(checkpoint, name)
          torch.save(checkpoint2, name2)
          source_path = name
          source_path2 = name2
          destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'    # 要改
          destination_path2 = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'

          shutil.copy(source_path, destination_path)
          shutil.copy(source_path2, destination_path2)

          depth_name_list = []
          image_name_list = []
          depth_list = []
          image_list = []

'''

"\ndef create_dataset_save(first_path, second_path, epoch_size = 5000): # first is depth, second is image\n  depth_list = []\n  image_list = []\n  depth_name_list = []\n  image_name_list = []\n  count = 0\n  file_count = 8  #改\n\n  first_path_list1 = sorted(os.listdir(first_path))\n  second_path_list1 = sorted(os.listdir(second_path))\n  idx1_start = 109   #改\n\n  for idx1 in range(idx1_start, len(first_path_list1)):\n\n    first_seq1 = first_path_list1[idx1]\n    # if first_seq1 == '2011_09_28_drive_0090_sync': #改\n    #   continue\n    first_path2 = first_path + '/' + first_seq1 + '/proj_depth/groundtruth'\n    second_path2 = second_path + '/' + first_seq1\n    first_path_list2 = sorted(os.listdir(first_path2))\n\n    if idx1 == idx1_start:\n      idx2_start = 1  # 改\n    else:\n      idx2_start = 0\n    for idx2 in range(idx2_start, len(first_path_list2)): # image02, image03\n      first_seq2 = first_path_list2[idx2]\n      first_path3 = first_path2 + '/' + first_seq2\n      second_

In [ ]:
'''
first_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth'
second_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image'
create_dataset_save(first_path, second_path, epoch_size = 5000) # first is depth, second is image

from google.colab import runtime
runtime.unassign()
'''

"\nfirst_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth'\nsecond_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Image'\ncreate_dataset_save(first_path, second_path, epoch_size = 5000) # first is depth, second is image\n\nfrom google.colab import runtime\nruntime.unassign()\n"

In [ ]:
'''
image_name_list = checkpoint['depth_name']
depth_name_list = checkpoint['depth_name']
print(depth_name_list[4999])
'''

"\nimage_name_list = checkpoint['depth_name']\ndepth_name_list = checkpoint['depth_name']\nprint(depth_name_list[4999])\n"

In [ ]:
'''
# for finding the location where it breaked
path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth'
file = sorted(os.listdir(path))
for idx in range(len(file)):
  if file[idx] == '2011_09_28_drive_0184_sync':
    print(idx)

path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth/2011_09_28_drive_0184_sync/proj_depth/groundtruth'
file = sorted(os.listdir(path))
for idx in range(len(file)):
  if file[idx] == 'image_03':
    print(idx)

path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth/2011_09_28_drive_0184_sync/proj_depth/groundtruth/image_03'
file = sorted(os.listdir(path))
for idx in range(len(file)):
  if file[idx] == '0000000062.png':
    print(idx)
'''

"\n# for finding the location where it breaked\npath = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth'\nfile = sorted(os.listdir(path))\nfor idx in range(len(file)):\n  if file[idx] == '2011_09_28_drive_0184_sync':\n    print(idx)\n\npath = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth/2011_09_28_drive_0184_sync/proj_depth/groundtruth'\nfile = sorted(os.listdir(path))\nfor idx in range(len(file)):\n  if file[idx] == 'image_03':\n    print(idx)\n\npath = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/Depth/2011_09_28_drive_0184_sync/proj_depth/groundtruth/image_03'\nfile = sorted(os.listdir(path))\nfor idx in range(len(file)):\n  if file[idx] == '0000000062.png':\n    print(idx)\n"

In [ ]:
'''
def create_dataset(data_per_epoch, amount_from_file, path): #data_per_epoch / amount_from_file要是整數

  name_list = sorted(os.listdir(path))

  if data_per_epoch % amount_from_file != 0:
    print("error, data_per_epoch can't divide amount_from_file !!!!!")
    sys.exit(1)

  file_idx = data_per_epoch // amount_from_file

  output_depth = []
  output_image = []
  length = len(name_list)
  for idx in range(file_idx):
    rand_int = random.randint(0, length - 1)
    tmp_file = name_list[rand_int]
    name = path + '/' + tmp_file
    checkpoint = torch.load(name)
    depth_list = checkpoint['depth_list']
    image_list = checkpoint['image_list']
    length_inside = len(depth_list)
    random_number = torch.randint(0, length_inside, (amount_from_file, ))

    for jdx in range(amount_from_file):

      output_depth.append(depth_list[random_number[jdx]])
      output_image.append(image_list[random_number[jdx]])
  return output_image, output_depth

'''

'\ndef create_dataset(data_per_epoch, amount_from_file, path): #data_per_epoch / amount_from_file要是整數\n\n  name_list = sorted(os.listdir(path))\n\n  if data_per_epoch % amount_from_file != 0:\n    print("error, data_per_epoch can\'t divide amount_from_file !!!!!")\n    sys.exit(1)\n\n  file_idx = data_per_epoch // amount_from_file\n\n  output_depth = []\n  output_image = []\n  length = len(name_list)\n  for idx in range(file_idx):\n    rand_int = random.randint(0, length - 1)\n    tmp_file = name_list[rand_int]\n    name = path + \'/\' + tmp_file\n    checkpoint = torch.load(name)\n    depth_list = checkpoint[\'depth_list\']\n    image_list = checkpoint[\'image_list\']\n    length_inside = len(depth_list)\n    random_number = torch.randint(0, length_inside, (amount_from_file, ))\n\n    for jdx in range(amount_from_file):\n\n      output_depth.append(depth_list[random_number[jdx]])\n      output_image.append(image_list[random_number[jdx]])\n  return output_image, output_depth\n\n'

In [ ]:
'''
def create_dataset_for_debug(data_per_epoch, amount_from_file, path): #data_per_epoch / amount_from_file要是整數
  name_list = sorted(os.listdir(path))
  file_idx = data_per_epoch // amount_from_file
  output_depth_path = []
  output_image_path = []
  output_depth = []
  output_image = []
  sym = 0
  if data_per_epoch % amount_from_file  != 0:
    file_idx = file_idx + 1
    sym = 1
  length = len(name_list)
  for idx in range(file_idx):
    rand_int = random.randint(0, length - 1)
    tmp_file = name_list[rand_int]
    name = path + '/' + tmp_file
    checkpoint = torch.load(name)
    depth_list = checkpoint['depth_list']
    image_list = checkpoint['image_list']
    length_inside = len(depth_list)
    if sym == 1:
      if idx == file_idx - 1:

        random_number = torch.randint(0, length_inside, ((data_per_epoch % amount_from_file), ))
      else:

        random_number = torch.randint(0, length_inside, (amount_from_file, ))


    else:
      random_number = torch.randint(0, length_inside, (amount_from_file, ))
    for jdx in range(amount_from_file):

      output_depth.append(depth_list[random_number[jdx]])
      output_image.append(image_list[random_number[jdx]])
  return output_image, output_depth

'''

"\ndef create_dataset_for_debug(data_per_epoch, amount_from_file, path): #data_per_epoch / amount_from_file要是整數\n  name_list = sorted(os.listdir(path))\n  file_idx = data_per_epoch // amount_from_file\n  output_depth_path = []\n  output_image_path = []\n  output_depth = []\n  output_image = []\n  sym = 0\n  if data_per_epoch % amount_from_file  != 0:\n    file_idx = file_idx + 1\n    sym = 1\n  length = len(name_list)\n  for idx in range(file_idx):\n    rand_int = random.randint(0, length - 1)\n    tmp_file = name_list[rand_int]\n    name = path + '/' + tmp_file\n    checkpoint = torch.load(name)\n    depth_list = checkpoint['depth_list']\n    image_list = checkpoint['image_list']\n    length_inside = len(depth_list)\n    if sym == 1:\n      if idx == file_idx - 1:\n\n        random_number = torch.randint(0, length_inside, ((data_per_epoch % amount_from_file), ))\n      else:\n\n        random_number = torch.randint(0, length_inside, (amount_from_file, ))\n\n\n    else:\n      random_n

In [ ]:
def create_dataset_large_epoch(random_list, now, data_path, name_path): #data_per_epoch / amount_from_file要是整數

  data_list = sorted(os.listdir(data_path))
  name_list = sorted(os.listdir(name_path))

  # file_idx = data_per_epoch // amount_from_file
  output_depth_path = []
  output_image_path = []
  output_depth = []
  output_image = []

  now_number = random_list[now]
  data_path = data_path + '/' + data_list[now_number]
  name_path = name_path + '/' + name_list[now_number]
  data_checkpoint = torch.load(data_path)
  name_checkpoint = torch.load(name_path)
  output_image_path = name_checkpoint['image_name']
  output_depth_path = name_checkpoint['depth_name']
  output_depth = data_checkpoint['depth_list']
  output_image = data_checkpoint['image_list']
  return output_image_path, output_depth_path, output_depth, output_image

In [ ]:
'''
data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
random_list = []

check_length = len(sorted(os.listdir(data_path)))
for idx in range(1, check_length + 1):
  random_list.append(idx)
random.shuffle(random_list)
now = 2 # 實際上now也要從0一路走到最大值


output_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)
'''

"\ndata_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'\nname_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'\nrandom_list = []\n\ncheck_length = len(sorted(os.listdir(data_path)))\nfor idx in range(1, check_length + 1):\n  random_list.append(idx)\nrandom.shuffle(random_list)\nnow = 2 # 實際上now也要從0一路走到最大值\n\n\noutput_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)\n"

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, img, depth):
        self.img = img
        self.depth = depth


    def __len__(self):
        return len(self.img)

    def __getitem__(self, idx):
        sample = {'img': self.img[idx], 'depth': self.depth[idx]}
        return sample


In [ ]:
'''
custom_dataset = CustomDataset(output_image, output_depth)
train_size = int(0.9 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])

batch_size = 128


trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
'''

'\ncustom_dataset = CustomDataset(output_image, output_depth)\ntrain_size = int(0.9 * len(custom_dataset))\nval_size = len(custom_dataset) - train_size\ntrain_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])\n\nbatch_size = 128\n\n\ntrainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)\nvalidloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)\n'

In [ ]:
def image_loader_to_tensor(tensor):
  tensor = tensor.to(torch.float32)
  tensor = tensor / 255.0
  tensor = tensor * 2.0
  tensor = tensor - 1.0
  tensor = tensor.permute(0, 3, 1, 2)
  return tensor

In [ ]:
def depth_loader_to_tensor(tensor, DEPTH_MEAN, DEPTH_STD):



  nonzero_mask = tensor != 0
  zero_mask = tensor == 0
  # mean = tensor[nonzero_mask].mean()
  # std = tensor[nonzero_mask].std()
  result = (tensor[nonzero_mask] - DEPTH_MEAN) / DEPTH_STD
  tensor[nonzero_mask] = result
  tensor[zero_mask] = -1



  return tensor

In [ ]:
'''
for batch in trainloader:
  print(batch['img'].shape)
  print(batch['img'].dtype)
  print(torch.max(batch['img']))
  print(torch.min(batch['img']))
  print(torch.mean(batch['img'].to(torch.float32)))
  print(batch['depth'].shape)
  print(batch['depth'].dtype)
  print(torch.max(batch['depth']))
  print(torch.min(batch['depth']))
  print(torch.mean(batch['depth']))
  break
'''

"\nfor batch in trainloader:\n  print(batch['img'].shape)\n  print(batch['img'].dtype)\n  print(torch.max(batch['img']))\n  print(torch.min(batch['img']))\n  print(torch.mean(batch['img'].to(torch.float32)))\n  print(batch['depth'].shape)\n  print(batch['depth'].dtype)\n  print(torch.max(batch['depth']))\n  print(torch.min(batch['depth']))\n  print(torch.mean(batch['depth']))\n  break\n"

In [ ]:
# for batch in trainloader:

#   batch['img'] = image_loader_to_tensor(batch['img'])
#   batch['depth'] = depth_loader_to_tensor(batch['depth'], DEPTH_MEAN, DEPTH_STD)
#   break
# nonzero = batch['depth'] != -1
# print(batch['img'].shape)
# print(batch['img'].dtype)
# print(torch.max(batch['img']))
# print(torch.min(batch['img']))
# print(torch.mean(batch['img']))
# print(batch['depth'].shape)
# print(batch['depth'].dtype)
# print(torch.max(batch['depth'][nonzero]))
# print(torch.min(batch['depth'][nonzero]))
# print(torch.mean(batch['depth'][nonzero]))
# print(torch.std(batch['depth'][nonzero]))

In [ ]:
'''
def image_tensor_to_numpy(tensor):

    tensor = tensor.permute(0, 2, 3, 1)
    output = tensor.numpy()
    output = output + 1.0
    output = output / 2.0
    output = output * 255.0
    output = output.astype(np.uint8)

    return output
'''

'\ndef image_tensor_to_numpy(tensor):\n\n    tensor = tensor.permute(0, 2, 3, 1)\n    output = tensor.numpy()\n    output = output + 1.0\n    output = output / 2.0\n    output = output * 255.0\n    output = output.astype(np.uint8)\n\n    return output\n'

In [ ]:
# this is min max normalize
'''
def depth_folder_to_tensor(folder_path):

    images_list = []



    files = sorted(os.listdir(folder_path))

    for file in files:

        file_path = os.path.join(folder_path, file)
        depth = np.array(Image.open(file_path), dtype=np.int16)
        assert(np.max(depth) > 255)
        depth = cv2.resize(depth, target_size, cv2.INTER_LANCZOS4)
        depth = depth.astype(np.float16) / 256.0
        images_list.append(depth)



    img2 = np.stack(images_list, axis=0)
    tensor = torch.tensor(img2)

    mini = torch.min(tensor[tensor != 0])
    tensor = (tensor - mini) / (tensor.max() - mini)
    tensor = torch.where(tensor < 0, -1, tensor)






    return tensor
'''

'\ndef depth_folder_to_tensor(folder_path):\n\n    images_list = []\n\n\n\n    files = sorted(os.listdir(folder_path))\n\n    for file in files:\n\n        file_path = os.path.join(folder_path, file)\n        depth = np.array(Image.open(file_path), dtype=np.int16)\n        assert(np.max(depth) > 255)\n        depth = cv2.resize(depth, target_size, cv2.INTER_LANCZOS4)\n        depth = depth.astype(np.float16) / 256.0\n        images_list.append(depth)\n\n\n\n    img2 = np.stack(images_list, axis=0)\n    tensor = torch.tensor(img2)\n\n    mini = torch.min(tensor[tensor != 0])\n    tensor = (tensor - mini) / (tensor.max() - mini)\n    tensor = torch.where(tensor < 0, -1, tensor)\n\n\n\n\n\n\n    return tensor\n'

In [ ]:
'''
# this is z-score normalization

def depth_folder_to_tensor(folder_path):

    images_list = []



    files = sorted(os.listdir(folder_path))

    for file in files:

        file_path = os.path.join(folder_path, file)
        depth = np.array(Image.open(file_path), dtype=np.int16)
        assert(np.max(depth) > 255)
        depth = cv2.resize(depth, target_size, cv2.INTER_LANCZOS4)
        depth = depth.astype(np.float16) / 256.0
        images_list.append(depth)



    img2 = np.stack(images_list, axis=0)
    tensor = torch.tensor(img2)
    nonzero_mask = tensor != 0
    zero_mask = tensor == 0
    mean = tensor[nonzero_mask].mean()
    std = tensor[nonzero_mask].std()
    result = (tensor[nonzero_mask] - mean) / std
    tensor[nonzero_mask] = result
    tensor[zero_mask] = -1



    return tensor, mean, std
'''

'\n# this is z-score normalization\n\ndef depth_folder_to_tensor(folder_path):\n\n    images_list = []\n\n\n\n    files = sorted(os.listdir(folder_path))\n\n    for file in files:\n\n        file_path = os.path.join(folder_path, file)\n        depth = np.array(Image.open(file_path), dtype=np.int16)\n        assert(np.max(depth) > 255)\n        depth = cv2.resize(depth, target_size, cv2.INTER_LANCZOS4)\n        depth = depth.astype(np.float16) / 256.0\n        images_list.append(depth)\n\n\n\n    img2 = np.stack(images_list, axis=0)\n    tensor = torch.tensor(img2)\n    nonzero_mask = tensor != 0\n    zero_mask = tensor == 0\n    mean = tensor[nonzero_mask].mean()\n    std = tensor[nonzero_mask].std()\n    result = (tensor[nonzero_mask] - mean) / std\n    tensor[nonzero_mask] = result\n    tensor[zero_mask] = -1\n\n\n\n    return tensor, mean, std\n'

In [ ]:
def get_beta_schedule(beta_schedule, *, beta_start, beta_end, num_diffusion_timesteps):
    def sigmoid(x):
        return 1 / (np.exp(-x) + 1)

    if beta_schedule == "quad":
        betas = (
            np.linspace(
                beta_start ** 0.5,
                beta_end ** 0.5,
                num_diffusion_timesteps,
                dtype=np.float64,
            )
            ** 2
        )
    elif beta_schedule == "linear":
        betas = np.linspace(
            beta_start, beta_end, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "const":
        betas = beta_end * np.ones(num_diffusion_timesteps, dtype=np.float64)
    elif beta_schedule == "jsd":  # 1/T, 1/(T-1), 1/(T-2), ..., 1
        betas = 1.0 / np.linspace(
            num_diffusion_timesteps, 1, num_diffusion_timesteps, dtype=np.float64
        )
    elif beta_schedule == "sigmoid":
        betas = np.linspace(-6, 6, num_diffusion_timesteps)
        betas = sigmoid(betas) * (beta_end - beta_start) + beta_start
    else:
        raise NotImplementedError(beta_schedule)
    assert betas.shape == (num_diffusion_timesteps,)
    return betas

In [ ]:
def compute_alpha(beta, t): # t給tensor 一維的
    beta = torch.cat([torch.zeros(1).to(beta.device), beta], dim=0)
    a = (1 - beta).cumprod(dim=0).index_select(0, t + 1).view(-1, 1, 1, 1)
    return a


In [ ]:
def get_timestep_embedding(timesteps, embedding_dim):

    assert len(timesteps.shape) == 1

    half_dim = embedding_dim // 2
    emb = math.log(10000) / (half_dim - 1)
    emb = torch.exp(torch.arange(half_dim, dtype=torch.float32) * -emb)
    emb = emb.to(device=timesteps.device)
    emb = timesteps.float()[:, None] * emb[None, :]
    emb = torch.cat([torch.sin(emb), torch.cos(emb)], dim=1)
    if embedding_dim % 2 == 1:  # zero pad
        emb = torch.nn.functional.pad(emb, (0, 1, 0, 0))
    return emb

In [ ]:
'''
def Normalize(in_channels):
    return torch.nn.GroupNorm(num_groups=32, num_channels=in_channels, eps=1e-6, affine=True)
'''

'\ndef Normalize(in_channels):\n    return torch.nn.GroupNorm(num_groups=32, num_channels=in_channels, eps=1e-6, affine=True)\n'

In [ ]:
# for the batch normalization
def Normalize(input, channels, momentum = 0.1, epsilon = 1e-5):
  bn = nn.BatchNorm2d(channels, momentum = momentum, eps = epsilon)
  return bn(input)

In [ ]:
'''
def nonlinearity(x):

    return x*torch.sigmoid(x)
'''

'\ndef nonlinearity(x):\n\n    return x*torch.sigmoid(x)\n'

In [ ]:
# def nonlinearity(input, negative_slope = 0.1, inplace = False):
#   return torch.nn.functional.leaky_relu(input_tensor, negative_slope = negative_slope, inplace = inplace)

In [ ]:
class Upsample(nn.Module): # this
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            self.conv = torch.nn.Conv2d(in_channels,  # this conv let the size unchanged
                                        in_channels,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)

    def forward(self, x):
        x = torch.nn.functional.interpolate(
            x, scale_factor=2.0, mode="nearest") # double the size
        if self.with_conv:
            x = self.conv(x)
        return x

In [ ]:
class Downsample(nn.Module):
    def __init__(self, in_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            # no asymmetric padding in torch conv, must do it ourselves
            self.conv = torch.nn.Conv2d(in_channels,  # halves the size
                                        in_channels,
                                        kernel_size=3,
                                        stride=2,
                                        padding=0)

    def forward(self, x):
        if self.with_conv:
            pad = (0, 1, 0, 1)
            x = torch.nn.functional.pad(x, pad, mode="constant", value=0) # 此動作相當於在每個圖片的channel的右邊下面pad 0
            x = self.conv(x)
        else:
            x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        return x

In [ ]:
class ResnetBlock(nn.Module):
    def __init__(self, *, in_channels, out_channels=None, conv_shortcut=False,
                 dropout, temb_channels=512):
        super().__init__()
        self.temb_channels = temb_channels
        self.in_channels = in_channels
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
        # self.Lrelu = nonlinearity
        out_channels = in_channels if out_channels is None else out_channels
        self.out_channels = out_channels
        self.use_conv_shortcut = conv_shortcut

        self.norm1 =nn.BatchNorm2d(in_channels)     # 這裡上面define的Normalize有點像是class的感覺
        self.conv1 = torch.nn.Conv2d(in_channels, # size unchanged
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        self.temb_proj = torch.nn.Linear(temb_channels,
                                         out_channels)
        self.norm2 = nn.BatchNorm2d(out_channels)
        self.dropout = torch.nn.Dropout(dropout) # param為機率
        self.conv2 = torch.nn.Conv2d(out_channels, # size unchanged
                                     out_channels,
                                     kernel_size=3,
                                     stride=1,
                                     padding=1)
        if self.in_channels != self.out_channels:
            if self.use_conv_shortcut:
                self.conv_shortcut = torch.nn.Conv2d(in_channels,   # size unchanged
                                                     out_channels,
                                                     kernel_size=3,
                                                     stride=1,
                                                     padding=1)
            else:
                self.nin_shortcut = torch.nn.Conv2d(in_channels,    # size unchanged
                                                    out_channels,
                                                    kernel_size=1,
                                                    stride=1,
                                                    padding=0)

    def forward(self, x, temb):
        h = x
        h = self.norm1(h)    # normalize

        h = self.Lrelu(h)  # sigmoid
        h = self.conv1(h)    # channel become out_channel

        h = h + self.temb_proj(self.Lrelu(temb))[:, :, None, None] # 後面加入None增加空的維度，針對temb_proj(nonlinearity(temb))使用，使其可以跟h相加，但是是使用broadcasting的方式
        h = self.norm2(h)
        h = self.Lrelu(h)
        h = self.dropout(h)
        h = self.conv2(h)

        if self.in_channels != self.out_channels:  # 如果inchannel和outchannel不同需要把輸入值channel也調整成一樣，用上conv2D，若inchannel和outchannel一樣就直接加
            if self.use_conv_shortcut:
                x = self.conv_shortcut(x)
            else:
                x = self.nin_shortcut(x)

        return x+h

In [ ]:
class AttnBlock(nn.Module):
    def __init__(self, in_channels):
        super().__init__()
        self.in_channels = in_channels

        self.norm = nn.BatchNorm2d(in_channels)
        self.q = torch.nn.Conv2d(in_channels, # in_cha == out_cha and the kernel size = 1, and size unchanged
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.k = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.v = torch.nn.Conv2d(in_channels,
                                 in_channels,
                                 kernel_size=1,
                                 stride=1,
                                 padding=0)
        self.proj_out = torch.nn.Conv2d(in_channels,
                                        in_channels,
                                        kernel_size=1,
                                        stride=1,
                                        padding=0)

    def forward(self, x):
        h_ = x
        h_ = self.norm(h_)
        q = self.q(h_)
        k = self.k(h_)
        v = self.v(h_)

        # compute attention
        b, c, h, w = q.shape # (batch, channel, height, width)
        q = q.reshape(b, c, h*w)
        q = q.permute(0, 2, 1)   # b,hw,c.  # 此兩變換(reshape + permute)詳細情形如下格，簡單來說最外圈還是每一個data(一張照片)，
                                # 往內一圈則是把該資料的所有channel(rgb)的同個位置放在一起
        k = k.reshape(b, c, h*w)  # b,c,hw # 單一此變換則是外圈是data，向內一圈則是該data一個channel內所有的值
        w_ = torch.bmm(q, k)     # b,hw,hw    w[b,i,j]=sum_c q[b,i,c]k[b,c,j]
        # 注意這邊是q * k，是把不同channels的同個位置變成vector然後內積，這樣跟cnn最大的差別是cnn只會在同一個區塊做相關性，attention卻在channels中的每個位置會相互做相關性
        w_ = w_ * (int(c)**(-0.5)) # 為何不是 * (int(h * w) ** (-0.5))?
        w_ = torch.nn.functional.softmax(w_, dim=2)

        # attend to values
        v = v.reshape(b, c, h*w)
        w_ = w_.permute(0, 2, 1)   # b,hw,hw (first hw of k, second of q)
        # b, c,hw (hw of q) h_[b,c,j] = sum_i v[b,c,i] w_[b,i,j]
        h_ = torch.bmm(v, w_)
        h_ = h_.reshape(b, c, h, w)

        h_ = self.proj_out(h_)

        return x+h_

In [ ]:
class DownsampleFPN(nn.Module):
    def __init__(self, in_channels, out_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            # no asymmetric padding in torch conv, must do it ourselves
            self.conv = torch.nn.Conv2d(in_channels,  # halves the size
                                        out_channels,
                                        kernel_size=3,
                                        stride=2,
                                        padding=0)

    def forward(self, x):
        if self.with_conv:
            pad = (0, 1, 0, 1)
            x = torch.nn.functional.pad(x, pad, mode="constant", value=0) # 此動作相當於在每個圖片的channel的右邊下面pad 0
            x = self.conv(x)
        else:
            x = torch.nn.functional.avg_pool2d(x, kernel_size=2, stride=2)
        return x

In [ ]:
class UpsampleFPN(nn.Module):
    def __init__(self, in_channels, out_channels, with_conv):
        super().__init__()
        self.with_conv = with_conv
        if self.with_conv:
            self.conv = torch.nn.Conv2d(int(in_channels),  # this conv let the size unchanged
                                        int(out_channels),
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)
    def forward(self, x):
        x = torch.nn.functional.interpolate(
            x, scale_factor=2.0, mode="nearest") # double the size
        if self.with_conv:
            x = self.conv(x)

        return x

In [ ]:
class FPN(nn.Module):  # 此處預設每次的resolutions都是上一次的一半 第一次的resolution是原圖的 1/4
    def __init__(self, config):
        super().__init__()
        resolutions = config['model']['FPN_conv_res'].copy()

        # resolutions = [64, 128, 256, 512]
        self.resolutions = resolutions.copy()

        # self.target_channel = int(resolutions[0] / 2)
        self.target_channel = config['model']['FPN_target_C']

        resolutions.insert(0, 3)
        # self.resolutions = resolutions # which is list
        self.ConvList = nn.ModuleList()
        self.tuneChannels = nn.ModuleList()
        self.Upsampple = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
        # self.Lrelu = nonlinearity
        self.bn0 = nn.BatchNorm2d(self.resolutions[0])
        for idx in range(len(resolutions) - 1):
          self.ConvList.append(DownsampleFPN(resolutions[idx],
                                          resolutions[idx + 1],
                                          True))

          self.tuneChannels.append(torch.nn.Conv2d(resolutions[idx + 1],
                                                   self.target_channel,
                                                   kernel_size = 3,
                                                   stride = 1,
                                                   padding = 1))
          if idx != len(resolutions) - 2:
            self.Upsampple.append(Upsample(self.target_channel, True))

        self.convOut = torch.nn.Conv2d(self.target_channel,
                                      self.target_channel,
                                      kernel_size = 1,
                                      stride = 1,
                                      padding = 0)
        self.norm_seq = nn.ModuleList()
        for idx in range((len(self.resolutions) - 2) * 2 + 2):
          print("up idx is = {}".format(idx))
          self.norm_seq.append(nn.BatchNorm2d(self.target_channel))




    def forward(self, x):
        h = x
        FPN_list = []

        for idx in range(len(self.resolutions)):


          if idx == 0:
            h = self.pool(self.Lrelu(self.ConvList[idx](h)))
          else:
            h = self.Lrelu(self.ConvList[idx](temp))

          temp = h
          h = self.Lrelu(self.tuneChannels[idx](h))
          FPN_list.append(h)
        count = 0
        for idx in reversed(range(len(self.resolutions))):
          if idx == 0:
            hold = self.norm_seq[count](hold)
            count += 1
            hold = self.convOut(hold + self.norm_seq[count](FPN_list[idx]))
            break
          if idx == len(self.resolutions) - 1:
            hold = self.Upsampple[idx - 1](FPN_list[idx])
          else:
            hold = self.norm_seq[count](hold)
            count += 1

            hold = hold + self.norm_seq[count](FPN_list[idx])
            count = count + 1
            hold = self.Upsampple[idx - 1](hold)

        return hold


In [ ]:
class depth_phase1_block(nn.Module):
  def __init__(self, in_cha, out_cha):
    super().__init__()
    self.conv = DownsampleFPN(in_cha, out_cha, True)
    # self.Lrelu = nonlinearity
    self.bn = nn.BatchNorm2d(out_cha)
    self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
  def forward(self, depth):
    return self.Lrelu(self.bn(self.conv(depth)))

In [ ]:
class depth_phase2_block(nn.Module):
  def __init__(self, in_cha, out_cha):
    super().__init__()
    self.conv = torch.nn.Conv2d(in_cha,
                                out_cha,
                                kernel_size = 3,
                                stride = 1,
                                padding = 1)
    # self.Lrelu = nonlinearity
    self.bn = nn.BatchNorm2d(out_cha)
    self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
  def forward(self, depth):
    return self.Lrelu(self.bn(self.conv(depth)))

In [ ]:
class depth_encode(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config.copy()
        resolution = config['data']['image_size']
        self.targeted_size = self.config['model']['depth_enc_targeted_size']
        self.enc_channels = self.config['model']['depth_enc_channels'].copy()
        # targeted_size = 64
        # enc_channels = [4, 16, 64, 256]
        enc_channels = self.enc_channels.copy()
        enc_channels.insert(0, 1)

        phase1 = 0
        while True:
          phase1 = phase1 + 1
          resolution = resolution / 2
          if resolution == self.targeted_size:
            break
        phase2 = len(self.enc_channels) - phase1
        self.phase1_model = nn.ModuleList()
        self.phase2_model = nn.ModuleList()

        for idx in range(phase1):
          self.phase1_model.append(depth_phase1_block(enc_channels[idx],
                                                      enc_channels[idx + 1]))
        for idx in range(phase2):
          self.phase2_model.append(depth_phase2_block(enc_channels[phase1 + idx],
                                                      enc_channels[phase1 + idx + 1]))

    def forward(self, depth):
        h = depth.unsqueeze(1)

        for idx in range(len(self.phase1_model)):
          h = self.phase1_model[idx](h)

        for idx in range(len(self.phase2_model)):
          h = self.phase2_model[idx](h)

        return h


In [ ]:
class depth_decode(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config.copy()
        self.ch = config['model']['ch']
        # self.ch = 128
        self.resolution = config['data']['image_size']
        # self.resolution = 256
        self.targeted_size = self.config['model']['depth_enc_targeted_size']
        # self.targeted_size = 64
        count = 0
        tmp = self.targeted_size
        while True:
          if tmp == self.resolution:
            break
          count = count + 1
          tmp = tmp * 2

        self.decode = nn.ModuleList()
        in_cha = self.ch
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
        # self.Lrelu = nonlinearity
        for idx in range(count):
          out_cha = in_cha / 4
          self.decode.append(UpsampleFPN(in_cha, out_cha, True))
          in_cha = out_cha
        self.final_conv = torch.nn.Conv2d(int(out_cha),  # this conv let the size unchanged
                                        1,
                                        kernel_size=3,
                                        stride=1,
                                        padding=1)








    def forward(self, pred):
        for idx in range(len(self.decode)):
          pred = self.decode[idx](pred)
        pred = self.final_conv(pred)
        pred = pred.squeeze(1)





        return pred

In [ ]:
def get_index_from_list(values, t, x_shape):
    batch_size = t.shape[0]
    """
    pick the values from vals
    according to the indices stored in `t`
    """
    result = values.gather(-1, t.cpu())
    """
    if
    x_shape = (5, 3, 64, 64)
        -> len(x_shape) = 4
        -> len(x_shape) - 1 = 3

    and thus we reshape `out` to dims
    (batch_size, 1, 1, 1)

    """
    return result.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

In [ ]:
class DiffusionModel:
    def __init__(self, beta_schedule = 'linear', start_schedule=0.0001, end_schedule=0.02, timesteps = 300):
        self.start_schedule = start_schedule
        self.end_schedule = end_schedule
        self.timesteps = timesteps

        """
        if
            betas = [0.1, 0.2, 0.3, ...]
        then
            alphas = [0.9, 0.8, 0.7, ...]
            alphas_cumprod = [0.9, 0.9 * 0.8, 0.9 * 0.8, * 0.7, ...]


        """
        betas = get_beta_schedule(beta_schedule, beta_start = start_schedule, beta_end = end_schedule, num_diffusion_timesteps = timesteps)
        self.betas = torch.tensor(betas)


        self.alphas = 1 - self.betas
        self.alphas_cumprod = torch.cumprod(self.alphas, axis=0)

    def forward(self, x_0, t, device):
        """
        x_0: (B, C, H, W)
        t: (B,)
        """


        noise = torch.randn_like(x_0)

        sqrt_alphas_cumprod_t = get_index_from_list(self.alphas_cumprod.sqrt(), t.to(torch.int64), x_0.shape)

        sqrt_one_minus_alphas_cumprod_t = get_index_from_list(torch.sqrt(1. - self.alphas_cumprod), t.to(torch.int64), x_0.shape)

        mean = sqrt_alphas_cumprod_t.to(device) * x_0.to(device)
        variance = sqrt_one_minus_alphas_cumprod_t.to(device) * noise.to(device)

        return mean + variance, noise.to(device) # mean為x_0乘以alpha bar, variance 為 noise 乘以 1-alpha bar


In [ ]:
class Model(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        ch, out_ch, ch_mult = config['model']['ch'], config['model']['out_ch'], tuple(config['model']['ch_mult'])
        num_res_blocks = config['model']['num_res_blocks']
        attn_resolutions = config['model']['attn_resolutions']
        dropout = config['model']['dropout']
        in_channels = config['model']['in_channels']
        resolution = config['data']['image_size']
        resamp_with_conv = config['model']['resamp_with_conv']
        num_timesteps = config['diffusion']['num_diffusion_timesteps']
        depth_enc_channels = config['model']['depth_enc_channels']
        if config['model']['type'] == 'bayesian':
            self.logvar = nn.Parameter(torch.zeros(num_timesteps))
        self.fpn = FPN(config)
        self.ch = ch

        self.temb_ch = self.ch*4
        self.num_resolutions = len(ch_mult)
        self.num_res_blocks = num_res_blocks
        self.resolution = resolution
        self.in_channels = in_channels

        # timestep embedding
        self.temb = nn.Module()
        self.temb.dense = nn.ModuleList([
            torch.nn.Linear(self.ch,
                            self.temb_ch),
            torch.nn.Linear(self.temb_ch,
                            self.temb_ch),
        ])

        '''
        # timestep embedding for diffusion---vvv
        self.temb.diff1 = nn.Linear(1, 1)
        self.temb.diff2 = nn.Linear(1, 1)
        # timestep embedding for diffusion---^^^
        '''


        self.depth_encode = depth_encode(config)

        # diffusion process ---vvv
        self.beta_schedule = config['diffusion']['beta_schedule']
        self.start_schedule = config['diffusion']['beta_start']
        self.end_schedule = config['diffusion']['beta_end']
        self.timesteps = config['diffusion']['num_diffusion_timesteps']
        self.diffusion_process = DiffusionModel(self.beta_schedule, self.start_schedule, self.end_schedule, self.timesteps)
        # diffusion process ---^^^



        # downsampling
        self.conv_in = torch.nn.Conv2d(depth_enc_channels[-1] * 2,
                                       self.ch,
                                       kernel_size=3,
                                       stride=1,
                                       padding=1)
# nonlinear

        curr_res = resolution
        in_ch_mult = (1,)+ch_mult
        self.down = nn.ModuleList()
        block_in = None
        for i_level in range(self.num_resolutions):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_in = ch*in_ch_mult[i_level]
            block_out = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks):

                block.append(ResnetBlock(in_channels=block_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            down = nn.Module()
            down.block = block
            down.attn = attn
            if i_level != self.num_resolutions-1:
                down.downsample = Downsample(block_in, resamp_with_conv)
                curr_res = curr_res // 2
            self.down.append(down)

        # middle
        self.mid = nn.Module()
        self.mid.block_1 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)
        self.mid.attn_1 = AttnBlock(block_in)
        self.mid.block_2 = ResnetBlock(in_channels=block_in,
                                       out_channels=block_in,
                                       temb_channels=self.temb_ch,
                                       dropout=dropout)

        # upsampling
        self.up = nn.ModuleList()
        for i_level in reversed(range(self.num_resolutions)):
            block = nn.ModuleList()
            attn = nn.ModuleList()
            block_out = ch*ch_mult[i_level]
            skip_in = ch*ch_mult[i_level]
            for i_block in range(self.num_res_blocks+1):
                if i_block == self.num_res_blocks:
                    skip_in = ch*in_ch_mult[i_level] # 最後一個block時inchannel數可能變少
                block.append(ResnetBlock(in_channels=block_in+skip_in,
                                         out_channels=block_out,
                                         temb_channels=self.temb_ch,
                                         dropout=dropout))
                block_in = block_out
                if curr_res in attn_resolutions:
                    attn.append(AttnBlock(block_in))
            up = nn.Module()
            up.block = block
            up.attn = attn
            if i_level != 0:
                up.upsample = Upsample(block_in, resamp_with_conv)
                curr_res = curr_res * 2
            self.up.insert(0, up)  # prepend to get consistent order, (0, up)代表在ModuleList()的0位置插入up這個Module

        # end
        self.norm_out = nn.BatchNorm2d(block_in)

        self.depth_decode = depth_decode(config)
        self.Lrelu = nn.LeakyReLU(negative_slope=0.1)
    def forward(self, image, depth, t):
        # assert x.shape[2] == x.shape[3] == self.resolution # to check if the height and width are the same with the resolution

        # timestep embedding
        temb = get_timestep_embedding(t, self.ch).to(device)
        temb = self.temb.dense[0](temb)
        temb = self.Lrelu(temb)
        temb = self.temb.dense[1](temb)



        img_enc = self.fpn(image)
        depth = self.depth_encode(depth)


        # depth = depth.unsqueeze(1)
        # return img_enc, depth

        # diffusion process ---vvv

        noisy_map, noise = self.diffusion_process.forward(depth, t, device = t.device)
        noisy_map = noisy_map.to(torch.float32)
        noise = noise.to(torch.float32)


        # diffusion process ---^^^


        # concat img_enc and noisy_map
        backbone_input = torch.cat([noisy_map, img_enc], dim = 1)
        # return backbone_input



        # downsampling down 裡面有好幾個元素，每個元素包含block(resblock), attn, downsample，其中attn只有幾個元素會有，downsample除了最後一個元素以外都有
        # 整個流程就是把x送進conv2D 然後送進down裡面經過resblock和部份attn downsample(conv2D)
        # hs = [self.conv_in(image)]

        hs = [self.conv_in(backbone_input)]

        for i_level in range(self.num_resolutions):
            for i_block in range(self.num_res_blocks):

                h = self.down[i_level].block[i_block](hs[-1], temb) # h 如果可以是attn就是attn 不然就是res, note that h是把值喂進去模塊後的值，要把hs的最後跟time embedded送進去
                if len(self.down[i_level].attn) > 0:
                    h = self.down[i_level].attn[i_block](h)
                hs.append(h)
            if i_level != self.num_resolutions-1:
                hs.append(self.down[i_level].downsample(hs[-1]))

        # middle
        h = hs[-1]
        h = self.mid.block_1(h, temb)
        h = self.mid.attn_1(h)
        h = self.mid.block_2(h, temb)

        # upsampling
        for i_level in reversed(range(self.num_resolutions)):
            for i_block in range(self.num_res_blocks+1):
                h = self.up[i_level].block[i_block](torch.cat([h, hs.pop()], dim=1), temb) # u-net的cat down
                if len(self.up[i_level].attn) > 0:
                    h = self.up[i_level].attn[i_block](h)
            if i_level != 0:
                h = self.up[i_level].upsample(h)

        # end
        h = self.norm_out(h)
        h = self.Lrelu(h)
        h = self.depth_decode(h)
        return h

In [ ]:
model = Model(config)
model = model.to(device)

up idx is = 0
up idx is = 1
up idx is = 2
up idx is = 3
up idx is = 4
up idx is = 5


In [ ]:
'''
# analysis for model weights
path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save/weight_{}.pth'.format(113)
checkpoint = torch.load(path, map_location = device)
'''

"\n# analysis for model weights\npath = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save/weight_{}.pth'.format(113)\ncheckpoint = torch.load(path, map_location = device)\n"

In [ ]:
'''
# analysis for model weights
weights = checkpoint['model_state_dict']
'''


"\n# analysis for model weights\nweights = checkpoint['model_state_dict']\n"

In [ ]:
'''
# analysis for model weights
print("number of keys : {}".format(len(weights)))
for key, value in weights.items():
    print(key, "\t", value.shape, "min = {}".format(torch.min(value)))
    print("------------")
'''

'\n# analysis for model weights\nprint("number of keys : {}".format(len(weights)))\nfor key, value in weights.items():\n    print(key, "\t", value.shape, "min = {}".format(torch.min(value)))\n    print("------------")\n'

In [ ]:
# BATCH_SIZE = 256

NO_LARGE_EPOCHS = 10
save_frequency = 5
LR = 0.001
VERBOSE = False
# data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
# name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
data_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/data_zip'
name_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/name_zip'
batch_size = 32
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
'''
# debug01 build dataset

random_list = []
check_length = len(sorted(os.listdir(data_path)))
for idx in range(1, check_length + 1):
  random_list.append(idx)
now = 0
random.shuffle(random_list)
output_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)

custom_dataset = CustomDataset(output_image, output_depth)
train_size = int(0.9 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
del train_dataset, val_dataset, custom_dataset
'''

'\n# debug01 build dataset\n\nrandom_list = []\ncheck_length = len(sorted(os.listdir(data_path)))\nfor idx in range(1, check_length + 1):\n  random_list.append(idx)\nnow = 0\nrandom.shuffle(random_list)\noutput_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)\n\ncustom_dataset = CustomDataset(output_image, output_depth)\ntrain_size = int(0.9 * len(custom_dataset))\nval_size = len(custom_dataset) - train_size\ntrain_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])\ntrainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)\nvalidloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)\ndel train_dataset, val_dataset, custom_dataset\n'

In [ ]:
'''
# debug02
for batch in trainloader:
  # print(batch.keys())
  t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
  input_img = batch['img'].to(torch.float32).to(device)
  input_img = image_loader_to_tensor(input_img)
  target_depth = batch['depth'].to(torch.float32).to(device)
  target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)
  # print(t)
  # print(input_img.shape)
  # print(torch.max(input_img))
  # print(torch.max(target_depth[0]))
  pred_depth = model(input_img, target_depth, t)
  break
'''

"\n# debug02\nfor batch in trainloader:\n  # print(batch.keys())\n  t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)\n  input_img = batch['img'].to(torch.float32).to(device)\n  input_img = image_loader_to_tensor(input_img)\n  target_depth = batch['depth'].to(torch.float32).to(device)\n  target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)\n  # print(t)\n  # print(input_img.shape)\n  # print(torch.max(input_img))\n  # print(torch.max(target_depth[0]))\n  pred_depth = model(input_img, target_depth, t)\n  break\n"

In [ ]:
# debug 04
# 接著可以每個dataset取一些值出來變成一個dataset來train
# data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
# name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
'''
data_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/data_zip'
name_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/name_zip'
data_list = sorted(os.listdir(data_path))

new_image_list = []
new_depth_list = []
new_image_name_list = []
new_depth_name_list = []
data_list_length = len(data_list)
for idx in range(data_list_length):
  check = torch.load(data_path + '/' + data_list[idx])
  check_name = torch.load(name_path + '/' + data_list[idx])
  for jdx in range(294):
    random_number = random.randint(0, 4999)
    new_image_list.append(check['image_list'][random_number])
    new_depth_list.append(check['depth_list'][random_number])
    new_image_name_list.append(check_name['image_name'][random_number])
    new_depth_name_list.append(check_name['depth_name'][random_number])

checkpoint = {
              'depth_list': new_depth_list,
              'image_list' : new_image_list,
          }
checkpoint2 = {
              'depth_name' : new_depth_name_list,
              'image_name' : new_image_name_list
          }

name = 'dataset_new.pth'
name2 = 'name_list_new.pth'
torch.save(checkpoint, name)
torch.save(checkpoint2, name2)

source_path = name
source_path2 = name2
destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'    # 要改
destination_path2 = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
shutil.copy(source_path, destination_path)
shutil.copy(source_path2, destination_path2)
'''

"\ndata_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/data_zip'\nname_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/name_zip'\ndata_list = sorted(os.listdir(data_path))\n\nnew_image_list = []\nnew_depth_list = []\nnew_image_name_list = []\nnew_depth_name_list = []\ndata_list_length = len(data_list)\nfor idx in range(data_list_length):\n  check = torch.load(data_path + '/' + data_list[idx])\n  check_name = torch.load(name_path + '/' + data_list[idx])\n  for jdx in range(294):\n    random_number = random.randint(0, 4999)\n    new_image_list.append(check['image_list'][random_number])\n    new_depth_list.append(check['depth_list'][random_number])\n    new_image_name_list.append(check_name['image_name'][random_number])\n    new_depth_name_list.append(check_name['depth_name'][random_number])\n\ncheckpoint = {\n              'depth_list': new_depth_list,\n              'image_list' : new_image_list,\n          }\ncheckpoint2 = {\n              'depth_na

In [ ]:
# debug 04
# 還有寫繼續train在大epoch還有哪些部分沒train完畢
# first epoch
epoch = 0
# data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
# name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
data_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Data/data_zip/dataset_new.pth'




check = torch.load(data_path)
output_depth = check['depth_list']
output_image = check['image_list']
custom_dataset = CustomDataset(output_image, output_depth)
train_size = int(0.9 * len(custom_dataset))
val_size = len(custom_dataset) - train_size
train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)


for now in range(100): # 一個小epoch是一個checkpoint檔，紀錄一次

  epoch += 1


  start_time = time.time()
  mean_epoch_loss = []
  mean_epoch_loss_val = []
  epoch_gradient = {}
  for batch in trainloader:
      t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

      input_img = batch['img'].to(torch.float32).to(device)
      input_img = image_loader_to_tensor(input_img)
      target_depth = batch['depth'].to(torch.float32).to(device)
      target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)

      pred_depth = model(input_img, target_depth, t)

      optimizer.zero_grad()
      loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
      mean_epoch_loss.append(loss.item())
      loss.backward()
      optimizer.step()

      # for name, param in model.named_pn arameters():
      #   if name not in epoch_gradient:
      #     epoch_gradient[name] = param.grad.clone()
      #   else:
      #     epoch_gradient[name] += param.grad
  with torch.inference_mode():
    for batch in validloader:
      t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
      input_img = batch['img'].to(torch.float32).to(device)

      input_img = image_loader_to_tensor(input_img)
      target_depth = batch['depth'].to(torch.float32).to(device)
      target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)
      pred_depth = model(input_img, target_depth, t)

      val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
      mean_epoch_loss_val.append(val_loss.item())

  if epoch % save_frequency == 0 :
    checkpoint = {
      'epoch': epoch,
      'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
      'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
      'valid_loss' : np.mean(mean_epoch_loss_val),
      'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
      # 'gradients' : epoch_gradient
    }

    torch.save(checkpoint, 'weight_{}.pth'.format(epoch))
    source_path = 'weight_{}.pth'.format(epoch)
    destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/model1_weight_debug04'


    # save them to the google drive
    shutil.copy(source_path, destination_path)

  #---計算時間---vvv
  end_time = time.time()
  exe_time = end_time - start_time
  hours, remainder = divmod(exe_time, 3600)
  minutes, seconds = divmod(remainder, 60)
  #---計算時間---^^^

  #-----以下是存loss的---vvv
  checkpoint = {
    'epoch': epoch,
    'valid_loss' : np.mean(mean_epoch_loss_val),
    'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
    # 'time' : exe_time
  }

  torch.save(checkpoint, 'loss_{}.pth'.format(epoch))
  source_path = 'loss_{}.pth'.format(epoch)
  destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/model1_loss_debug04'


  # save them to the google drive
  shutil.copy(source_path, destination_path)
  #-----以下是存loss的---^^^

  print('---')
  print(f"Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
  print("time = {}:{}:{}".format(hours, minutes, seconds))


---
Epoch: 1 | Train Loss 0.49356757679155894 | Val Loss 0.4185864965120951
time = 0.0:2.0:28.972631216049194
---
Epoch: 2 | Train Loss 0.4025674879550934 | Val Loss 0.3932868798573812
time = 0.0:2.0:29.553516387939453
---
Epoch: 3 | Train Loss 0.3905226579734257 | Val Loss 0.3907417853673299
time = 0.0:2.0:29.566131830215454
---
Epoch: 4 | Train Loss 0.3817241621868951 | Val Loss 0.38105852007865904
time = 0.0:2.0:29.52786874771118
---
Epoch: 5 | Train Loss 0.3698875901954515 | Val Loss 0.366618146498998
time = 0.0:2.0:58.72800421714783
---
Epoch: 6 | Train Loss 0.36634673731667655 | Val Loss 0.3631379723548889
time = 0.0:2.0:30.235275506973267
---
Epoch: 7 | Train Loss 0.36133480817079544 | Val Loss 0.3597558875878652
time = 0.0:2.0:29.53512692451477
---
Epoch: 8 | Train Loss 0.3564714510525976 | Val Loss 0.3534411648909251
time = 0.0:2.0:29.8062903881073
---
Epoch: 9 | Train Loss 0.3542402682559831 | Val Loss 0.35052496790885923
time = 0.0:2.0:29.923146963119507
---
Epoch: 10 | Trai

In [ ]:
# debug 03
# 思考LOSS這樣的原因
# 可能是因為每個EPOCH自身的資料太相近
# 而EPOCH和EPOCH之間的資料又差距太大
# 試試看只對某個PTH做會發生什麼事情
epoch = 0
# data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
# name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
check_length = len(sorted(os.listdir(data_path)))

random_list = []
large_epoch = 1
for idx in range(1, check_length + 1):
  random_list.append(idx)

# random.shuffle(random_list)

for now in range(100): # 一個小epoch是一個checkpoint檔，紀錄一次

  epoch += 1
  if now == 0:
    output_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, 0, data_path, name_path)
  custom_dataset = CustomDataset(output_image, output_depth)
  train_size = int(0.9 * len(custom_dataset))
  val_size = len(custom_dataset) - train_size
  train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
  trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
  validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
  start_time = time.time()
  mean_epoch_loss = []
  mean_epoch_loss_val = []
  epoch_gradient = {}
  for batch in trainloader:
      t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

      input_img = batch['img'].to(torch.float32).to(device)
      input_img = image_loader_to_tensor(input_img)
      target_depth = batch['depth'].to(torch.float32).to(device)
      target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)

      pred_depth = model(input_img, target_depth, t)

      optimizer.zero_grad()
      loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
      mean_epoch_loss.append(loss.item())
      loss.backward()
      optimizer.step()

      # for name, param in model.named_pn arameters():
      #   if name not in epoch_gradient:
      #     epoch_gradient[name] = param.grad.clone()
      #   else:
      #     epoch_gradient[name] += param.grad
  with torch.inference_mode():
    for batch in validloader:
      t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
      input_img = batch['img'].to(torch.float32).to(device)

      input_img = image_loader_to_tensor(input_img)
      target_depth = batch['depth'].to(torch.float32).to(device)
      target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)
      pred_depth = model(input_img, target_depth, t)

      val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
      mean_epoch_loss_val.append(val_loss.item())

  if epoch % save_frequency == 0 or epoch == check_length * NO_LARGE_EPOCHS:
    checkpoint = {
      'large_epoch' : large_epoch,
      'epoch': epoch,
      'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
      'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
      'valid_loss' : np.mean(mean_epoch_loss_val),
      'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
      'now' : now
      # 'gradients' : epoch_gradient
    }

    torch.save(checkpoint, 'weight_{}_{}.pth'.format(large_epoch, epoch))
    source_path = 'weight_{}_{}.pth'.format(large_epoch, epoch)
    destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save1'


    # save them to the google drive
    shutil.copy(source_path, destination_path)

  #---計算時間---vvv
  end_time = time.time()
  exe_time = end_time - start_time
  hours, remainder = divmod(exe_time, 3600)
  minutes, seconds = divmod(remainder, 60)
  #---計算時間---^^^

  #-----以下是存loss的---vvv
  checkpoint = {
    'large_epoch' : large_epoch,
    'epoch': epoch,
    'valid_loss' : np.mean(mean_epoch_loss_val),
    'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
    # 'time' : exe_time
  }

  torch.save(checkpoint, 'loss_{}_{}.pth'.format(large_epoch, epoch))
  source_path = 'loss_{}_{}.pth'.format(large_epoch, epoch)
  destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save1'


  # save them to the google drive
  shutil.copy(source_path, destination_path)
  #-----以下是存loss的---^^^

  print('---')
  print(f"Large Epoch: {large_epoch}, Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
  print("time = {}:{}:{}".format(hours, minutes, seconds))


---
Large Epoch: 1, Epoch: 1 | Train Loss 0.49343522403921397 | Val Loss 0.4880920072396596
time = 0.0:2.0:36.17216897010803
---
Large Epoch: 1, Epoch: 2 | Train Loss 0.46241805787597384 | Val Loss 0.44704047838846844
time = 0.0:2.0:41.61445426940918
---
Large Epoch: 1, Epoch: 3 | Train Loss 0.43448203078338077 | Val Loss 0.4255649824937185
time = 0.0:2.0:41.41892766952515
---
Large Epoch: 1, Epoch: 4 | Train Loss 0.42302542754581995 | Val Loss 0.4182444016138713
time = 0.0:2.0:41.69097113609314
---
Large Epoch: 1, Epoch: 5 | Train Loss 0.4146978112203734 | Val Loss 0.408508958419164
time = 0.0:2.0:57.86175298690796
---
Large Epoch: 1, Epoch: 6 | Train Loss 0.41101817403520857 | Val Loss 0.4117868423461914
time = 0.0:2.0:41.74451184272766
---
Large Epoch: 1, Epoch: 7 | Train Loss 0.4078986849103655 | Val Loss 0.39844924012819927
time = 0.0:2.0:41.30629348754883
---
Large Epoch: 1, Epoch: 8 | Train Loss 0.4028422081044742 | Val Loss 0.3999204834302266
time = 0.0:2.0:41.77875900268555
--

KeyboardInterrupt: 

In [ ]:
# 還有寫繼續train在大epoch還有哪些部分沒train完畢
# first epoch
epoch = 0
# data_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/data_zip'
# name_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Data/name_zip'
check_length = len(sorted(os.listdir(data_path)))
for large_epoch in range(1, NO_LARGE_EPOCHS + 1):

    random_list = []

    for idx in range(1, check_length + 1):
        random_list.append(idx)

    random.shuffle(random_list)

    for now in range(check_length): # 一個小epoch是一個checkpoint檔，紀錄一次

        epoch += 1
        output_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)
        custom_dataset = CustomDataset(output_image, output_depth)
        train_size = int(0.9 * len(custom_dataset))
        val_size = len(custom_dataset) - train_size
        train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
        trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
        validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
        start_time = time.time()
        mean_epoch_loss = []
        mean_epoch_loss_val = []
        epoch_gradient = {}
        for batch in trainloader:
            t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

            input_img = batch['img'].to(torch.float32).to(device)
            input_img = image_loader_to_tensor(input_img)
            target_depth = batch['depth'].to(torch.float32).to(device)
            target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)

            pred_depth = model(input_img, target_depth, t)

            optimizer.zero_grad()
            loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
            mean_epoch_loss.append(loss.item())
            loss.backward()
            optimizer.step()

            # for name, param in model.named_pn arameters():
            #   if name not in epoch_gradient:
            #     epoch_gradient[name] = param.grad.clone()
            #   else:
            #     epoch_gradient[name] += param.grad
        with torch.inference_mode():
            for batch in validloader:
                t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
                input_img = batch['img'].to(torch.float32).to(device)

                input_img = image_loader_to_tensor(input_img)
                target_depth = batch['depth'].to(torch.float32).to(device)
                target_depth = depth_loader_to_tensor(target_depth, DEPTH_MEAN, DEPTH_STD)
                pred_depth = model(input_img, target_depth, t)

                val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
                mean_epoch_loss_val.append(val_loss.item())

        if epoch % save_frequency == 0 or epoch == check_length * NO_LARGE_EPOCHS:
            checkpoint = {
                'large_epoch' : large_epoch,
                'epoch': epoch,
                'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
                'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
                'valid_loss' : np.mean(mean_epoch_loss_val),
                'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
                'now' : now
                # 'gradients' : epoch_gradient
            }

            torch.save(checkpoint, 'weight_{}_{}.pth'.format(large_epoch, epoch))
            source_path = 'weight_{}_{}.pth'.format(large_epoch, epoch)
            destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save1'


            # save them to the google drive
            shutil.copy(source_path, destination_path)

            #---計算時間---vvv
            end_time = time.time()
            exe_time = end_time - start_time
            hours, remainder = divmod(exe_time, 3600)
            minutes, seconds = divmod(remainder, 60)
            #---計算時間---^^^

            #-----以下是存loss的---vvv
            checkpoint = {
            'large_epoch' : large_epoch,
            'epoch': epoch,
            'valid_loss' : np.mean(mean_epoch_loss_val),
            'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
            # 'time' : exe_time
            }

            torch.save(checkpoint, 'loss_{}_{}.pth'.format(large_epoch, epoch))
            source_path = 'loss_{}_{}.pth'.format(large_epoch, epoch)
            destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save1'


            # save them to the google drive
            shutil.copy(source_path, destination_path)
            #-----以下是存loss的---^^^

        print('---')
        print(f"Large Epoch: {large_epoch}, Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
        print("time = {}:{}:{}".format(hours, minutes, seconds))



---
Large Epoch: 1, Epoch: 1 | Train Loss 0.13994502164423467 | Val Loss 0.10218588908513387
time = 0.0:2.0:37.41733717918396
---
Large Epoch: 1, Epoch: 2 | Train Loss 0.5529141611286572 | Val Loss 0.5212518314520518
time = 0.0:2.0:26.342310667037964
---
Large Epoch: 1, Epoch: 3 | Train Loss 0.6504759699106216 | Val Loss 0.641002341111501
time = 0.0:2.0:22.769662618637085


In [ ]:
large_epoch =
epoch =
load_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Checkpoint/weight_save1/weight_{}_{}.pth'.format(large_epoch, epoch)
data_path = ''
check_length = len(sorted(os.listdir(data_path)))
checkpoint = torch.load(load_path)
if (checkpoint['now'].item() == check_length - 1):
    large_epoch_start = large_epoch + 1
    del large_epoch

    for large_epoch in range(large_epoch_start, NO_LARGE_EPOCHS + 1):
        random_list = []
        for idx in range(check_length):
            random_list.append(idx)
        random.shuffle(random_list)

        for now in range(check_length): # 一個小epoch是一個checkpoint檔，紀錄一次

            epoch += 1
            output_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)
            custom_dataset = CustomDataset(output_image, output_depth)
            train_size = int(0.9 * len(custom_dataset))
            val_size = len(custom_dataset) - train_size
            train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
            trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
            validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
            start_time = time.time()
            mean_epoch_loss = []
            mean_epoch_loss_val = []
            epoch_gradient = {}
            for batch in trainloader:
                t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

                input_img = batch['img'].to(torch.float32).to(device)
                target_depth = batch['depth'].to(torch.float32).to(device)

                pred_depth = model(input_img, target_depth, t)

                optimizer.zero_grad()
                loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
                mean_epoch_loss.append(loss.item())
                loss.backward()
                optimizer.step()

                # for name, param in model.named_parameters():
                #   if name not in epoch_gradient:
                #     epoch_gradient[name] = param.grad.clone()
                #   else:
                #     epoch_gradient[name] += param.grad
            with torch.inference_mode():
                for batch in validloader:
                    t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
                    input_img = batch['img'].to(torch.float32).to(device)
                    target_depth = batch['depth'].to(torch.float32).to(device)

                    pred_depth = model(input_img, target_depth, t)

                    val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
                    mean_epoch_loss_val.append(val_loss.item())

            if epoch % save_frequency == 0 or epoch == check_length * NO_LARGE_EPOCHS:
                checkpoint = {
                    'large_epoch' : large_epoch,
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
                    'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
                    'valid_loss' : np.mean(mean_epoch_loss_val),
                    'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
                    'now' : now,
                    'random_list' : random_list
                    # 'gradients' : epoch_gradient
                }

                torch.save(checkpoint, 'weight_{}_{}.pth'.format(large_epoch, epoch))
                source_path = 'weight_{}_{}.pth'.format(large_epoch, epoch)
                destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save1'


                # save them to the google drive
                shutil.copy(source_path, destination_path)

                #---計算時間---vvv
                end_time = time.time()
                exe_time = end_time - start_time
                hours, remainder = divmod(exe_time, 3600)
                minutes, seconds = divmod(remainder, 60)
                #---計算時間---^^^

                #-----以下是存loss的---vvv
                checkpoint = {
                'large_epoch' : large_epoch,
                'epoch': epoch,
                'valid_loss' : np.mean(mean_epoch_loss_val),
                'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
                'time' : exe_time
                }

                torch.save(checkpoint, 'loss_{}_{}.pth'.format(large_epoch, epoch))
                source_path = 'loss_{}_{}.pth'.format(large_epoch, epoch)
                destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save1'


                # save them to the google drive
                shutil.copy(source_path, destination_path)
                #-----以下是存loss的---^^^

            print('---')
            print(f"Large Epoch: {large_epoch}, Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
            print("time = {}:{}:{}".format(hours, minutes, seconds))

else:
    large_epoch_start = large_epoch
    del large_epoch

    for largh_epoch in range(large_epoch_start, NO_LARGE_EPOCHS + 1):
        if large_epoch == large_epoch_start:
            now_start = checkpoint['now'].item() + 1
            random_list = checkpoint['random_list']
        else:
            now_start = 0
            random_list = []
            for idx in range(check_length):
                random_list.append(idx)
            random.shuffle(random_list)


        for now in range(now_start, check_length):
            epoch += 1
            output_image_path, output_depth_path, output_depth, output_image = create_dataset_large_epoch(random_list, now, data_path, name_path)
            custom_dataset = CustomDataset(output_image, output_depth)
            train_size = int(0.9 * len(custom_dataset))
            val_size = len(custom_dataset) - train_size
            train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])
            trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
            validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
            start_time = time.time()
            mean_epoch_loss = []
            mean_epoch_loss_val = []
            epoch_gradient = {}
            for batch in trainloader:
                t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

                input_img = batch['img'].to(torch.float32).to(device)
                target_depth = batch['depth'].to(torch.float32).to(device)

                pred_depth = model(input_img, target_depth, t)

                optimizer.zero_grad()
                loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
                mean_epoch_loss.append(loss.item())
                loss.backward()
                optimizer.step()

                # for name, param in model.named_parameters():
                #   if name not in epoch_gradient:
                #     epoch_gradient[name] = param.grad.clone()
                #   else:
                #     epoch_gradient[name] += param.grad
            with torch.inference_mode():
                for batch in validloader:
                    t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
                    input_img = batch['img'].to(torch.float32).to(device)
                    target_depth = batch['depth'].to(torch.float32).to(device)

                    pred_depth = model(input_img, target_depth, t)

                    val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
                    mean_epoch_loss_val.append(val_loss.item())

            if epoch % save_frequency == 0 or epoch == check_length * NO_LARGE_EPOCHS:
                checkpoint = {
                    'large_epoch' : large_epoch,
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
                    'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
                    'valid_loss' : np.mean(mean_epoch_loss_val),
                    'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
                    'now' : now,
                    'random_list' : random_list
                    # 'gradients' : epoch_gradient
                }

                torch.save(checkpoint, 'weight_{}_{}.pth'.format(large_epoch, epoch))
                source_path = 'weight_{}_{}.pth'.format(large_epoch, epoch)
                destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save1'


                # save them to the google drive
                shutil.copy(source_path, destination_path)

                #---計算時間---vvv
                end_time = time.time()
                exe_time = end_time - start_time
                hours, remainder = divmod(exe_time, 3600)
                minutes, seconds = divmod(remainder, 60)
                #---計算時間---^^^

                #-----以下是存loss的---vvv
                checkpoint = {
                'large_epoch' : large_epoch,
                'epoch': epoch,
                'valid_loss' : np.mean(mean_epoch_loss_val),
                'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
                'time' : exe_time
                }

                torch.save(checkpoint, 'loss_{}_{}.pth'.format(large_epoch, epoch))
                source_path = 'loss_{}_{}.pth'.format(large_epoch, epoch)
                destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save1'


                # save them to the google drive
                shutil.copy(source_path, destination_path)
                #-----以下是存loss的---^^^

            print('---')
            print(f"Large Epoch: {large_epoch}, Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
            print("time = {}:{}:{}".format(hours, minutes, seconds))




In [ ]:
test = torch.tensor([1]).item()
test == 1


True

In [ ]:
# first epoch
'''



for epoch in range(1, NO_EPOCHS + 1, 1):
    start_time = time.time()
    mean_epoch_loss = []
    mean_epoch_loss_val = []
    epoch_gradient = {}
    for batch in trainloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        optimizer.zero_grad()
        loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss.append(loss.item())
        loss.backward()


        for name, param in model.named_parameters():
          if name not in epoch_gradient:
            epoch_gradient[name] = param.grad.clone()
          else:
            epoch_gradient[name] += param.grad
        optimizer.step()

    with torch.inference_mode():
      for batch in validloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss_val.append(val_loss.item())

    if epoch % PRINT_FREQUENCY == 0 or epoch == NO_EPOCHS:
        checkpoint = {
          'epoch': epoch,
          'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
          'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'gradients' : epoch_gradient
        }

        torch.save(checkpoint, 'weight_{}.pth'.format(epoch))
        source_path = 'weight_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save'


        # save them to the google drive
        shutil.copy(source_path, destination_path)

        #---計算時間---vvv
        end_time = time.time()
        exe_time = end_time - start_time
        hours, remainder = divmod(execution_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        #---計算時間---^^^

        #-----以下是存loss的---vvv
        checkpoint = {
          'epoch': epoch,
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'time' : exe_time
        }

        torch.save(checkpoint, 'loss_{}.pth'.format(epoch))
        source_path = 'loss_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save'


        # save them to the google drive
        shutil.copy(source_path, destination_path)
        #-----以下是存loss的---^^^

        print('---')
        print(f"Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
        print("time = {}:{}:{}".format(hours, minutes, seconds))
'''

In [ ]:



# train_size = int(0.9 * len(custom_dataset))
# val_size = len(custom_dataset) - train_size


# train_dataset, val_dataset = random_split(custom_dataset, [train_size, val_size])

# # 使用DataLoader加载数据集
# batch_size = 32
# trainloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last = True)
# validloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True, drop_last = True)

In [ ]:
# print(input_img.shape)
# print(target_depth.shape)
# print(t.shape)
# count = 0
# for batch in trainloader:
#   count = count + 1
# print(count)
# print(32 * 31)

In [ ]:
def training(start_epoch, steps, load_path, model, PRINT_FREQUENCY,  optimizer, save_frequency, trainloader, validloader, datasets_path, batch_size, config, device, weights_save_path, loss_save_path, gradient_save_path = None, save_gradient = False): # load path is where the already existed weights save
  initial = start_epoch

  NO_EPOCHS = steps # 要多做幾個epochs
  # load_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Checkpoint/weight_save/weight_{}.pth'.format(initial) # 位置要改
  checkpoint = torch.load(load_path)

  start = checkpoint['epoch'] + 1
  # model_state_dict = checkpoint['model_state_dict']

  model.load_state_dict(checkpoint['model_state_dict'])
  optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
  for epoch in range(start , start + NO_EPOCHS + 1):
      start_time = time.time()
      epoch_gradient = {}
      mean_epoch_loss = []
      mean_epoch_loss_val = []
      for batch in trainloader:
          t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

          input_img = batch['img'].to(torch.float32).to(device)
          target_depth = batch['depth'].to(torch.float32).to(device)

          pred_depth = model(input_img, target_depth, t)

          optimizer.zero_grad()
          loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
          mean_epoch_loss.append(loss.item())
          loss.backward()

          if save_gradient :
            for name, param in model.named_parameters():
              if name not in epoch_gradient:
                epoch_gradient[name] = param.grad.clone()
              else:
                epoch_gradient[name] += param.grad


          optimizer.step()

      with torch.inference_mode():
        for batch in validloader:
          t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
          input_img = batch['img'].to(torch.float32).to(device)
          target_depth = batch['depth'].to(torch.float32).to(device)

          pred_depth = model(input_img, target_depth, t)

          val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
          mean_epoch_loss_val.append(val_loss.item())

      if epoch % save_frequency == 0 or epoch == start + NO_EPOCHS:
          checkpoint = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
            'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
            'valid_loss' : np.mean(mean_epoch_loss_val),
            'loss' : np.mean(mean_epoch_loss) # 記得不能存tensor
          }

          torch.save(checkpoint, 'weight_{}.pth'.format(epoch))
          source_path = 'weight_{}.pth'.format(epoch)
          destination_path = weights_save_path


          # save them to the google drive
          shutil.copy(source_path, destination_path)
          #---計算時間---vvv
          end_time = time.time()
          exe_time = end_time - start_time
          hours, remainder = divmod(execution_time, 3600)
          minutes, seconds = divmod(remainder, 60)
          #---計算時間---^^^
          #-----以下是存loss的---vvv
          checkpoint = {
            'epoch': epoch,
            'valid_loss' : np.mean(mean_epoch_loss_val),
            'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
            'time' : exe_time
          }

          torch.save(checkpoint, 'loss_{}.pth'.format(epoch))
          source_path = 'loss_{}.pth'.format(epoch)
          destination_path = loss_save_path
          #-----存gradient---vvv
          if save_gradient:
            checkpoint = {
            'gradients' : epoch_gradient
          }
          torch.save(checkpoint, 'gradient_{}.pth'.format(epoch))
          source_path = 'gradient_{}.pth'.format(epoch)
          destination_path = gradient_save_path
          shutil.copy(source_path, destination_path)

          #-----存gradient---^^^
          # save them to the google drive
          shutil.copy(source_path, destination_path)
          #-----以下是存loss的---^^^

        if epoch % PRINT_FREQUENCY == 0:
          print('---')
          print(f"Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
          print("time = {}:{}:{}".format(hours, minutes, seconds))


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 94)

In [ ]:
# first epoch
'''
import shutil


for epoch in range(1, NO_EPOCHS + 1, 1):
    start_time = time.time()
    mean_epoch_loss = []
    mean_epoch_loss_val = []
    epoch_gradient = {}
    for batch in trainloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        optimizer.zero_grad()
        loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss.append(loss.item())
        loss.backward()


        for name, param in model.named_parameters():
          if name not in epoch_gradient:
            epoch_gradient[name] = param.grad.clone()
          else:
            epoch_gradient[name] += param.grad
        optimizer.step()

    with torch.inference_mode():
      for batch in validloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss_val.append(val_loss.item())

    if epoch % PRINT_FREQUENCY == 0 or epoch == NO_EPOCHS:
        checkpoint = {
          'epoch': epoch,
          'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
          'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'gradients' : epoch_gradient
        }

        torch.save(checkpoint, 'weight_{}.pth'.format(epoch))
        source_path = 'weight_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save'


        # save them to the google drive
        shutil.copy(source_path, destination_path)

        #---計算時間---vvv
        end_time = time.time()
        exe_time = end_time - start_time
        hours, remainder = divmod(execution_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        #---計算時間---^^^

        #-----以下是存loss的---vvv
        checkpoint = {
          'epoch': epoch,
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'time' : exe_time
        }

        torch.save(checkpoint, 'loss_{}.pth'.format(epoch))
        source_path = 'loss_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save'


        # save them to the google drive
        shutil.copy(source_path, destination_path)
        #-----以下是存loss的---^^^

        print('---')
        print(f"Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
        print("time = {}:{}:{}".format(hours, minutes, seconds))
'''

'\nimport shutil\nfrom google.colab import files\n\nfor epoch in range(1, NO_EPOCHS + 1, 1):\n    mean_epoch_loss = []\n    mean_epoch_loss_val = []\n    epoch_gradient = {}\n    for batch in trainloader:\n        t = torch.randint(0, config[\'diffusion\'][\'num_diffusion_timesteps\'], (batch_size,)).long().to(device)\n\n        input_img = batch[\'img\'].to(torch.float32).to(device)\n        target_depth = batch[\'depth\'].to(torch.float32).to(device)\n\n        pred_depth = model(input_img, target_depth, t)\n\n        optimizer.zero_grad()\n        loss = torch.nn.functional.mse_loss(target_depth, pred_depth)\n        mean_epoch_loss.append(loss.item())\n        loss.backward()\n\n\n        for name, param in model.named_parameters():\n          if name not in epoch_gradient:\n            epoch_gradient[name] = param.grad.clone()\n          else:\n            epoch_gradient[name] += param.grad\n        optimizer.step()\n\n    with torch.inference_mode():\n      for batch in validload

In [ ]:
# import torch
# import shutil
# from google.colab import files
# initial = 16
# NO_EPOCHS = 200 # 要多做幾個epochs
# load_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save/weight_{}.pth'.format(initial) # 位置要改
# checkpoint = torch.load(load_path)

In [ ]:
# continue training
initial = 90
NO_EPOCHS = 20 # 要多做幾個epochs
load_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Checkpoint/weight_save/weight_{}.pth'.format(initial) # 位置要改
checkpoint = torch.load(load_path)

start = checkpoint['epoch'] + 1
# model_state_dict = checkpoint['model_state_dict']

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
for epoch in range(start , start + NO_EPOCHS + 1):
    start_time = time.time()
    epoch_gradient = {}
    mean_epoch_loss = []
    mean_epoch_loss_val = []
    for batch in trainloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        optimizer.zero_grad()
        loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss.append(loss.item())
        loss.backward()

        for name, param in model.named_parameters():
          if name not in epoch_gradient:
            epoch_gradient[name] = param.grad.clone()
          else:
            epoch_gradient[name] += param.grad


        optimizer.step()

    with torch.inference_mode():
      for batch in validloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss_val.append(val_loss.item())

    if epoch % PRINT_FREQUENCY == 0 or epoch == start + NO_EPOCHS:
        checkpoint = {
          'epoch': epoch,
          'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
          'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'gradients' : epoch_gradient
        }

        torch.save(checkpoint, 'weight_{}.pth'.format(epoch))
        source_path = 'weight_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save_gradient_dev'


        # save them to the google drive
        shutil.copy(source_path, destination_path)


        #---計算時間---vvv
        end_time = time.time()
        exe_time = end_time - start_time
        hours, remainder = divmod(execution_time, 3600)
        minutes, seconds = divmod(remainder, 60)
        #---計算時間---^^^
        #-----以下是存loss的---vvv
        checkpoint = {
          'epoch': epoch,
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'time' : exe_time
        }

        torch.save(checkpoint, 'loss_{}.pth'.format(epoch))
        source_path = 'loss_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save_gradient_dev'


        # save them to the google drive
        shutil.copy(source_path, destination_path)
        #-----以下是存loss的---^^^


        print('---')
        print(f"Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
        print("time = {}:{}:{}".format(hours, minutes, seconds))


---
Epoch: 91 | Train Loss 0.38567152300051283 | Val Loss 0.39286285638809204
---
Epoch: 92 | Train Loss 0.3859223084790366 | Val Loss 0.3921920657157898
---
Epoch: 93 | Train Loss 0.3854115701147488 | Val Loss 0.3909292121728261
---
Epoch: 94 | Train Loss 0.38506057752030237 | Val Loss 0.38761888941129047
---
Epoch: 95 | Train Loss 0.3845937837447439 | Val Loss 0.3917753994464874
---
Epoch: 96 | Train Loss 0.38600521215370726 | Val Loss 0.3998758594195048
---
Epoch: 97 | Train Loss 0.38719060378415243 | Val Loss 0.39039607842763263
---
Epoch: 98 | Train Loss 0.38408188096114565 | Val Loss 0.3900303343931834
---
Epoch: 99 | Train Loss 0.3831964518342699 | Val Loss 0.38861361145973206
---
Epoch: 100 | Train Loss 0.3895272814801761 | Val Loss 0.3920138378938039
---
Epoch: 101 | Train Loss 0.39488488116434645 | Val Loss 0.3970567186673482
---
Epoch: 102 | Train Loss 0.6200624565993037 | Val Loss 0.6198289394378662
---
Epoch: 103 | Train Loss 0.600775699530329 | Val Loss 0.6032946904500326

In [ ]:
'''
import torch
import os
for idx in range(133):
  if idx == 45:
    continue

  path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save/loss_{}.pth'.format(idx)
  check = torch.load(path)
  print("epoch : {}, loss = {}  | val_loss = {}".format(check['epoch'], check['loss'], check['valid_loss']))
  print('----------')

'''

'\nimport torch\nimport os\nfor idx in range(133):\n  if idx == 45:\n    continue\n\n  path = \'/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save/loss_{}.pth\'.format(idx)\n  check = torch.load(path)\n  print("epoch : {}, loss = {}  | val_loss = {}".format(check[\'epoch\'], check[\'loss\'], check[\'valid_loss\']))\n  print(\'----------\')\n\n'

In [ ]:
'''
# continue training
# train for another account
# change the lr

# continue training
# train for another account
# change the lr

import shutil
from google.colab import files
initial = 100
NO_EPOCHS = 2000 # 要多做幾個epochs
load_path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Checkpoint/weight_save_gradient_dev/weight_{}.pth'.format(initial) # 位置要改
checkpoint = torch.load(load_path)

start = checkpoint['epoch'] + 1
# model_state_dict = checkpoint['model_state_dict']

model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
for epoch in range(start , start + NO_EPOCHS + 1):
    for param_group in optimizer.param_groups:
      param_group['lr'] = 0.0001
    epoch_gradient = {}
    mean_epoch_loss = []
    mean_epoch_loss_val = []
    for batch in trainloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)

        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        optimizer.zero_grad()
        loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss.append(loss.item())
        loss.backward()

        for name, param in model.named_parameters():
          if name not in epoch_gradient:
            epoch_gradient[name] = param.grad.clone()
          else:
            epoch_gradient[name] += param.grad


        optimizer.step()

    with torch.inference_mode():
      for batch in validloader:
        t = torch.randint(0, config['diffusion']['num_diffusion_timesteps'], (batch_size,)).long().to(device)
        input_img = batch['img'].to(torch.float32).to(device)
        target_depth = batch['depth'].to(torch.float32).to(device)

        pred_depth = model(input_img, target_depth, t)

        val_loss = torch.nn.functional.mse_loss(target_depth, pred_depth)
        mean_epoch_loss_val.append(val_loss.item())

    if epoch % PRINT_FREQUENCY == 0 or epoch == start + NO_EPOCHS:
        checkpoint = {
          'epoch': epoch,
          'model_state_dict': model.state_dict(), # model.state_dict()是存下param的的值和形狀
          'optimizer_state_dict': optimizer.state_dict(), # optimizer.state_dict()則是存下優化器的param如momentum等等 不包含當下梯度
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss), # 記得不能存tensor
          'gradients' : epoch_gradient
        }

        torch.save(checkpoint, 'weight_{}.pth'.format(epoch))
        source_path = 'weight_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/weight_save'


        # save them to the google drive
        shutil.copy(source_path, destination_path)

        #-----以下是存loss的---vvv
        checkpoint = {
          'epoch': epoch,
          'valid_loss' : np.mean(mean_epoch_loss_val),
          'loss' : np.mean(mean_epoch_loss) # 記得不能存tensor
        }

        torch.save(checkpoint, 'loss_{}.pth'.format(epoch))
        source_path = 'loss_{}.pth'.format(epoch)
        destination_path = '/content/drive/MyDrive/Colab Notebooks/Simple_DE/Checkpoint/loss_save'


        # save them to the google drive
        shutil.copy(source_path, destination_path)
        #-----以下是存loss的---^^^


        print('---')
        print(f"Epoch: {epoch} | Train Loss {np.mean(mean_epoch_loss)} | Val Loss {np.mean(mean_epoch_loss_val)}")
'''

In [ ]:
'''
for idx in range(91, 111):
  path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Checkpoint/loss_save_gradient_dev/loss_{}.pth'.format(idx)
'''

In [ ]:
'''
# load gradient
epoch = 102
path = '/content/drive/MyDrive/Colab Notebooks/共用區/Simple_DE/Checkpoint/weight_save_gradient_dev/weight_{}.pth'.format(epoch)
check = torch.load(path, map_location = 'cpu')
'''

In [ ]:
'''
gradients = check['gradients']
torch.set_printoptions(precision=10)

path

for name, grad in gradients.items():
  # if name == 'fpn.ConvList.0.conv.bias':
    maximum = torch.max(grad)
    minimum = torch.min(grad)
    mean_val = torch.mean(grad)
    print(name, " : ", grad.shape, " | min = ", minimum, " | max = ", maximum, " | mean = ", mean_val)
    print("-----------------------")
    # break

torch.min(grad)
minimum

torch.set_printoptions(precision=6)
print(maximum)
'''

In [ ]:
from google.colab import runtime
runtime.unassign()